In [1]:
from dateutil.relativedelta import relativedelta

import os, netCDF4
import numpy as np
import pandas as pd
import xarray as xr
import scipy

import CRPS
import scipy.interpolate
from datetime import date, datetime
import rasterio, rioxarray, rasterstats, netCDF4
import xskillscore, os, pickle
from datetime import timedelta

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from multiprocessing import Pool
from itertools import repeat
import calendar

### Reading in raw Forecasts and placing them in a dictionary split by water year

In [17]:
cwd = os.getcwd()+"\RURC2"
dirs = os.listdir(cwd)

In [18]:
dfList = {}
for item in dirs:
    if "." not in item:
        dfList[item] = {}
        for _zip in os.listdir(cwd+"\\"+item):
            if ".gz" in _zip:
                file_name = cwd + "\\" + item + "\\" + _zip
                df = pd.read_csv(file_name,compression='gzip',header=14, sep=',', quotechar='"')
                df['Unnamed: 0']=df['Unnamed: 0'].str[:10]
                df = df.rename(columns={'Unnamed: 0':"Date"}).astype({'Date':'datetime64[ns]'}).set_index("Date")
                dfList[item][_zip[9:-7]] = df

In [19]:
dfList.keys()

dict_keys(['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'])

In [2]:
f_obsFlow = pd.read_excel(r"C:\Users\jmccaskill\Documents\Ruedi Reservoir Operations Pilot Study\5YearReforecasts_MonthlyTimestep-20221115T135645Z-001\5YearReforecasts_MonthlyTimestep\Reudi Reservoir Observed Flows.xlsx",parse_dates=True,index_col=0).astype(float)

In [3]:
CFSDAY2ACFT = 24 * 60 * 60 / 43560
ACFT2CFSDAY = 1 / CFSDAY2ACFT
CFSDAY2ACFT

1.9834710743801653

# streamflow.py

### Summation Logic:
- Check date of forecast
     - If date is on or before April 1st of that water year do not sum
     - If the date is on or between May 1st and July 31st add the observed sum from April 1st to the forecasted date of that given water year
     - Ex: forecast date 1991-07-01 is takes one month forecast and adds observed summation from 1991-04-01 to 1991-06-30 for all ensemble trace years
- The runoff period observed summation applied to all ensemble trace years in the forecast
    - Observed summation from previous/future years are not applied
    - Ex: forecast from 1995-07-01 will have the observed summation from 1995-04-01 to 1995-06-30 applied to all ensemble years in this forecast
- The weather trace year that is the same year as the forecast is dropped
    Ex: forecast from 1993-01-01 drops ensemble trace year 1993

### Assessment Logic:
- Forecasts are grouped by month 
    - Ex: (1990-10-01, 1991-10-01, 1992-10-01,...)
- 

In [4]:
def calculate_rfc_period(s_rfc_directory):
    # todo: doc string

    # Get the current files in the directory
    l_Files = os.listdir(s_rfc_directory)

    # Sort files alphanumerically
    l_Files.sort()
    sorted(l_Files)

    # Filter out non-CSV files
    csv_names = [] # create empty list
    for files in l_Files:
        if '.csv' in files:
            csv_names.append(files)

    # Get the first and last files
    first_file = csv_names[0]
    last_file = csv_names[-1]

    # Get year, month, and day of first/last file
    f_year = first_file[9:13]
    f_month = first_file[14:16]
    f_day = first_file[17:19]
    l_year = last_file[9:13]
    l_month = last_file[14:16]
    l_day = last_file[17:19]
    

    # Create timestamps
    o_rfc_start = datetime(year=int(f_year),month=int(f_month),day=int(f_day))
    o_rfc_end = datetime(year=int(l_year), month=int(l_month), day=int(l_day))

    return o_rfc_start, o_rfc_end


def calculate_obsflow_period(d_obs_flow):
    # todo: doc string

    # Create timestamps
    o_obsflow_start = d_obs_flow.index[0]
    o_obsflow_end = d_obs_flow.index[-1]

    return o_obsflow_start, o_obsflow_end



def _import_RFC(start_date,end_date,forecast_path, observed_data):
    forecast_dict = {}

    forecast_list = list()
    
    # loop through each monthly forecast
    for i, t in enumerate(pd.date_range(start=start_date,end=end_date,freq="MS")): 
        
        # add forecast months to dictionary
        if str(t.month) not in forecast_dict.keys():
            forecast_dict[str(t.month)]=[]
       
    
        #creating runoff_period
        if t.month > 7:
            runoff_period = pd.date_range(start=datetime(year=t.year+1,month=4,day=1),end=datetime(year=t.year+1, month=7, day=31))
        else:
            runoff_period = pd.date_range(start=datetime(year=t.year,month=4,day=1),end=datetime(year=t.year, month=7, day=31))
             
        if runoff_period[0].year > 2020:
            continue
        
        #calculating forecast sum for runoff period using forecasts generated outside runoff period
        #no addition of observational data 
        if t.month < 5 or t.month > 7: 

            # reading in file
            forecasts = pd.read_csv(forecast_path+"\\RURC2L_F."+str(t.year)+"-"+str(t.month).zfill(2)+"-"+str(t.day).zfill(2)+'.csv.gz',compression='gzip',header=14, sep=',', quotechar='"')
            
            #formatting file
            forecasts['Unnamed: 0']=forecasts['Unnamed: 0'].str[:10]
            forecasts = forecasts.rename(columns={'Unnamed: 0':"Date"}).set_index("Date")
            forecasts.index = pd.to_datetime(forecasts.index)

            #filtering forecast to only contain data in runoff period
            forecast = forecasts[forecasts.index.isin(runoff_period)]

            #sum forecast for each ensemble year
            forecast_sum = forecast.sum(axis=0)

            #save ensemble years to be used as coordinates in xarray
            cols = forecast_sum.index.astype(int)

            #generating xarray
            forecast_i = xr.DataArray(np.array(forecast_sum*CFSDAY2ACFT), coords=[cols],dims=["flow"])

            print(t,"No summation required")

            
            #add xarray to list in dictionary keyed for that month
            forecast_dict[str(t.month)].append(forecast_i)
            
        else:
            
            #creating observation period that will be added to forecast summation
            obs_period = pd.date_range(start=datetime(year=t.year,month=4,day=1),end=datetime(year=t.year, month=t.month-1, day=calendar.monthrange(t.year,t.month-1)[1]))
            
            #reading in forecast
            forecasts = pd.read_csv(forecast_path+"\\RURC2L_F."+str(t.year)+"-"+str(t.month).zfill(2)+"-"+str(t.day).zfill(2)+'.csv.gz',compression='gzip',header=14, sep=',', quotechar='"')
            
            #formatting file
            forecasts['Unnamed: 0']=forecasts['Unnamed: 0'].str[:10]
            forecasts = forecasts.rename(columns={'Unnamed: 0':"Date"}).set_index("Date")
            forecasts.index = pd.to_datetime(forecasts.index)
            
            #filtering forecast data to be only within runoff period
            forecast = forecasts[forecasts.index.isin(runoff_period)]

            #summation of observed data from April 1st to the forecast date (not including forecast date)
            obs = observed_data[observed_data.index.isin(obs_period)].sum()

            #summation of forecasted data
            forecast_sum = forecast.sum(axis=0)

            #checks
            print("---Forecasted Runoff Period", runoff_period[0], "to", runoff_period[-1],"---")
            print("---Observed Runoff Period",obs_period[0],"to",obs_period[-1],"---")
            print("1991 Forecast Sum:",forecast_sum[0], "+", obs, "=")
            
            #add sum of observed data to sum of forecasted data
            forecast_sum += obs
            print(forecast_sum[0])
                                          
            #saving ensemble years to be used as coordinates in xarray
            cols = forecast_sum.index.astype(int)

            #generating xarray
            forecast_i = xr.DataArray(np.array(forecast_sum*CFSDAY2ACFT), coords=[cols],dims=["flow"])

            #add xarray to list in dictionary keyed for that month
            forecast_dict[str(t.month)].append(forecast_i)

    #list of all dates a forecast was generated on        
    t = pd.date_range(start_date, end_date, freq='MS').to_pydatetime()
        

    return t, forecast_dict


def _import_RFC2(start_date,end_date,forecast_path, observed_data):
    forecast_dict_second_attempt = {}

    forecast_list = list()
    
    # loop through each monthly forecast
    for i, t in enumerate(pd.date_range(start=start_date,end=end_date,freq="MS")): 
        
        # add forecast months to dictionary
        if str(t.month) not in forecast_dict_second_attempt.keys():
            forecast_dict_second_attempt[str(t.month)]=[]
       
    
#         creating runoff_period
        if t.month > 7:
            runoff_period = pd.date_range(start=datetime(year=t.year+1,month=4,day=1),end=datetime(year=t.year+1, month=7, day=31))
        else:
            runoff_period = pd.date_range(start=datetime(year=t.year,month=4,day=1),end=datetime(year=t.year, month=7, day=31))
             
                
        if runoff_period[0].year > 2020:
            continue
        
        #calculating forecast sum for runoff period using forecasts generated outside runoff period
        #no addition of observational data 
#         if t.month < 5 or t.month > 7: 

        # reading in file
        forecasts = pd.read_csv(forecast_path+"\\RURC2L_F."+str(t.year)+"-"+str(t.month).zfill(2)+"-"+str(t.day).zfill(2)+'.csv.gz',compression='gzip',header=14, sep=',', quotechar='"')

        #formatting file
        forecasts['Unnamed: 0']=forecasts['Unnamed: 0'].str[:10]
        forecasts = forecasts.rename(columns={'Unnamed: 0':"Date"}).set_index("Date")
        forecasts.index = pd.to_datetime(forecasts.index)

        #filtering forecast to only contain data in runoff period
        forecast = forecasts[forecasts.index.isin(runoff_period)]

        #sum forecast for each ensemble year
        forecast_sum = forecast.sum(axis=0)

        #save ensemble years to be used as coordinates in xarray
        cols = forecast_sum.index.astype(int)

        #generating xarray
        forecast_i = xr.DataArray(np.array(forecast_sum*CFSDAY2ACFT), coords=[cols],dims=["flow"])

        print(t,"No summation required")


        #add xarray to list in dictionary keyed for that month
        forecast_dict_second_attempt[str(t.month)].append(forecast_i)
            
    #list of all dates a forecast was generated on        
    t = pd.date_range(start_date, end_date, freq='MS').to_pydatetime()
        

    return t, forecast_dict_second_attempt


if __name__ == "__main__":

    ### Specify the input data ###
    # Set the forecast information
    ia_issuance_hours = np.array([0])
    i_forecast_length = 10                      # [days]


    ### Set directory of RFC forecasts ###
    s_rfc_directory = r"C:\Users\jmccaskill\Documents\Ruedi Reservoir Operations Pilot Study\5YearReforecasts_MonthlyTimestep-20221115T135645Z-001\5YearReforecasts_MonthlyTimestep\RURC2"

    ### Specify observed streamflow data, in cfs
    s_obs_flow_directory = r"C:\Users\jmccaskill\Documents\Ruedi Reservoir Operations Pilot Study\5YearReforecasts_MonthlyTimestep-20221115T135645Z-001\5YearReforecasts_MonthlyTimestep\Reudi Reservoir Observed Flows.xlsx"
    da_obs_flow = pd.read_excel(s_obs_flow_directory, index_col=0, header=0).astype(float) # todo: put this in a function and have it return an xarray
    

    ### Set the days of interest to calculate the skill ###
#     ia_skill_periods = np.arange(0, 10, 1)           # [days]


    #####################################################################################################################################################################################
    ### Determine the period of time from each of the sources ###
    ## Determine the period of record from the RFC data ##
    o_rfc_start, o_rfc_end = calculate_rfc_period(s_rfc_directory)

    ## Determine the period of record from observed streamflow data ##
    o_obsflow_start, o_obsflow_end = calculate_obsflow_period(da_obs_flow)

    ## Determine which case is limiting to establish the analysis period ##
    # Set the start for the period of record coverage
    if o_rfc_start < o_obsflow_start:
        # Observed start is after GEFS start. Observed limits.
        o_por_start = o_obsflow_start
    else:
        # GEFS start is after the observed start. GEFS limits.
        o_por_start = o_rfc_start

    # Set the end for the period of record coverage
    if o_rfc_end < o_obsflow_end:
        # GEFS terminates before the observed. GEFS limits.
        o_por_end = o_rfc_end
    else:
        # Observed terminates before the GEFS. Observed limits.
        o_por_end = o_obsflow_end

        
    #calling primary function
    ol_rfc_dates, all_runoff_data = _import_RFC(o_rfc_start, o_rfc_end, s_rfc_directory, da_obs_flow.squeeze())
    ol_rfc_dates2, only_july_data = _import_RFC2(o_rfc_start, o_rfc_end, s_rfc_directory, da_obs_flow.squeeze())

    #creating observation xarray
    dx_obsflow = da_obs_flow.squeeze()[(da_obs_flow.index.month > 3) & (da_obs_flow.index.month < 8) & (da_obs_flow.index.year < 2021)]
    dx_obsflow = dx_obsflow.groupby(dx_obsflow.index.year).sum()*CFSDAY2ACFT
    dx_obsflow = dx_obsflow.to_xarray().rename({"        DATETIME":"ensemble_year"})


1990-10-01 00:00:00 No summation required
1990-11-01 00:00:00 No summation required
1990-12-01 00:00:00 No summation required
1991-01-01 00:00:00 No summation required
1991-02-01 00:00:00 No summation required
1991-03-01 00:00:00 No summation required
1991-04-01 00:00:00 No summation required
---Forecasted Runoff Period 1991-04-01 00:00:00 to 1991-07-31 00:00:00 ---
---Observed Runoff Period 1991-04-01 00:00:00 to 1991-04-30 00:00:00 ---
1991 Forecast Sum: 59448.33 + 2631.8409999999994 =
62080.171
---Forecasted Runoff Period 1991-04-01 00:00:00 to 1991-07-31 00:00:00 ---
---Observed Runoff Period 1991-04-01 00:00:00 to 1991-05-31 00:00:00 ---
1991 Forecast Sum: 42828.1 + 18873.116999999987 =
61701.21699999999
---Forecasted Runoff Period 1991-04-01 00:00:00 to 1991-07-31 00:00:00 ---
---Observed Runoff Period 1991-04-01 00:00:00 to 1991-06-30 00:00:00 ---
1991 Forecast Sum: 11989.52 + 51056.20499999997 =
63045.72499999998
1991-08-01 00:00:00 No summation required
1991-09-01 00:00:00 No 

1999-04-01 00:00:00 No summation required
---Forecasted Runoff Period 1999-04-01 00:00:00 to 1999-07-31 00:00:00 ---
---Observed Runoff Period 1999-04-01 00:00:00 to 1999-04-30 00:00:00 ---
1991 Forecast Sum: 59082.38 + 2648.2879999999977 =
61730.668
---Forecasted Runoff Period 1999-04-01 00:00:00 to 1999-07-31 00:00:00 ---
---Observed Runoff Period 1999-04-01 00:00:00 to 1999-05-31 00:00:00 ---
1991 Forecast Sum: 52693.41 + 17394.077999999994 =
70087.488
---Forecasted Runoff Period 1999-04-01 00:00:00 to 1999-07-31 00:00:00 ---
---Observed Runoff Period 1999-04-01 00:00:00 to 1999-06-30 00:00:00 ---
1991 Forecast Sum: 16207.03 + 49966.64599999989 =
66173.67599999989
1999-08-01 00:00:00 No summation required
1999-09-01 00:00:00 No summation required
1999-10-01 00:00:00 No summation required
1999-11-01 00:00:00 No summation required
1999-12-01 00:00:00 No summation required
2000-01-01 00:00:00 No summation required
2000-02-01 00:00:00 No summation required
2000-03-01 00:00:00 No summati

2007-10-01 00:00:00 No summation required
2007-11-01 00:00:00 No summation required
2007-12-01 00:00:00 No summation required
2008-01-01 00:00:00 No summation required
2008-02-01 00:00:00 No summation required
2008-03-01 00:00:00 No summation required
2008-04-01 00:00:00 No summation required
---Forecasted Runoff Period 2008-04-01 00:00:00 to 2008-07-31 00:00:00 ---
---Observed Runoff Period 2008-04-01 00:00:00 to 2008-04-30 00:00:00 ---
1991 Forecast Sum: 89734.65000000001 + 3930.256249999995 =
93664.90625
---Forecasted Runoff Period 2008-04-01 00:00:00 to 2008-07-31 00:00:00 ---
---Observed Runoff Period 2008-04-01 00:00:00 to 2008-05-31 00:00:00 ---
1991 Forecast Sum: 84466.12999999999 + 25271.928437599974 =
109738.05843759996
---Forecasted Runoff Period 2008-04-01 00:00:00 to 2008-07-31 00:00:00 ---
---Observed Runoff Period 2008-04-01 00:00:00 to 2008-06-30 00:00:00 ---
1991 Forecast Sum: 29499.75 + 69323.93802099998 =
98823.68802099998
2008-08-01 00:00:00 No summation required
20

2015-08-01 00:00:00 No summation required
2015-09-01 00:00:00 No summation required
2015-10-01 00:00:00 No summation required
2015-11-01 00:00:00 No summation required
2015-12-01 00:00:00 No summation required
2016-01-01 00:00:00 No summation required
2016-02-01 00:00:00 No summation required
2016-03-01 00:00:00 No summation required
2016-04-01 00:00:00 No summation required
---Forecasted Runoff Period 2016-04-01 00:00:00 to 2016-07-31 00:00:00 ---
---Observed Runoff Period 2016-04-01 00:00:00 to 2016-04-30 00:00:00 ---
1991 Forecast Sum: 59802.11 + 3919.4986295999925 =
63721.60862959999
---Forecasted Runoff Period 2016-04-01 00:00:00 to 2016-07-31 00:00:00 ---
---Observed Runoff Period 2016-04-01 00:00:00 to 2016-05-31 00:00:00 ---
1991 Forecast Sum: 50170.11 + 17597.505637599985 =
67767.61563759999
---Forecasted Runoff Period 2016-04-01 00:00:00 to 2016-07-31 00:00:00 ---
---Observed Runoff Period 2016-04-01 00:00:00 to 2016-06-30 00:00:00 ---
1991 Forecast Sum: 9481.240000000002 + 5

1996-12-01 00:00:00 No summation required
1997-01-01 00:00:00 No summation required
1997-02-01 00:00:00 No summation required
1997-03-01 00:00:00 No summation required
1997-04-01 00:00:00 No summation required
1997-05-01 00:00:00 No summation required
1997-06-01 00:00:00 No summation required
1997-07-01 00:00:00 No summation required
1997-08-01 00:00:00 No summation required
1997-09-01 00:00:00 No summation required
1997-10-01 00:00:00 No summation required
1997-11-01 00:00:00 No summation required
1997-12-01 00:00:00 No summation required
1998-01-01 00:00:00 No summation required
1998-02-01 00:00:00 No summation required
1998-03-01 00:00:00 No summation required
1998-04-01 00:00:00 No summation required
1998-05-01 00:00:00 No summation required
1998-06-01 00:00:00 No summation required
1998-07-01 00:00:00 No summation required
1998-08-01 00:00:00 No summation required
1998-09-01 00:00:00 No summation required
1998-10-01 00:00:00 No summation required
1998-11-01 00:00:00 No summation r

2013-11-01 00:00:00 No summation required
2013-12-01 00:00:00 No summation required
2014-01-01 00:00:00 No summation required
2014-02-01 00:00:00 No summation required
2014-03-01 00:00:00 No summation required
2014-04-01 00:00:00 No summation required
2014-05-01 00:00:00 No summation required
2014-06-01 00:00:00 No summation required
2014-07-01 00:00:00 No summation required
2014-08-01 00:00:00 No summation required
2014-09-01 00:00:00 No summation required
2014-10-01 00:00:00 No summation required
2014-11-01 00:00:00 No summation required
2014-12-01 00:00:00 No summation required
2015-01-01 00:00:00 No summation required
2015-02-01 00:00:00 No summation required
2015-03-01 00:00:00 No summation required
2015-04-01 00:00:00 No summation required
2015-05-01 00:00:00 No summation required
2015-06-01 00:00:00 No summation required
2015-07-01 00:00:00 No summation required
2015-08-01 00:00:00 No summation required
2015-09-01 00:00:00 No summation required
2015-10-01 00:00:00 No summation r

### Creating 10 data arrays consisting of forecasts corresponding to the 10 months of each year that are used for the analysis

In [5]:
# Reordering months to place August and September in front of October
desired_order_list = ['8', '9','10', '11', '12', '1', '2', '3', '4', '5', '6', '7']

only_july_data_reordered = {k: only_july_data[k] for k in desired_order_list}
all_runoff_data_reordered = {k: all_runoff_data[k] for k in desired_order_list}

In [6]:
all_runoff_data_reordered.keys()

dict_keys(['8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7'])

#### Making Xarrays so data is easier to parse through

In [7]:
# Xarray for originial methodology
dx_forecasts_all=[]
for i in all_runoff_data_reordered.keys():
    if int(i) >= 10:
        start_year=1990
    else:
        start_year=1991
    dates=[]
    if int(i) == 8 or int(i) == 9:
        for j in range(29):
            dates.append(datetime(year=start_year+j,month=int(i),day=1))
    else:
        for j in range(30):
            dates.append(datetime(year=start_year+j,month=int(i),day=1))

    dx_forecasts_all.append(xr.DataArray(data=all_runoff_data[i],coords=[dates,all_runoff_data[i][0].flow],dims=['forecast_date','ensemble_year']))

In [8]:
# Xarray for second methodology
dx_forecasts_second_attempt=[]
for i in only_july_data_reordered.keys():
    if int(i) >= 10:
        start_year=1990
    else:
        start_year=1991
    dates=[]
    if int(i) == 8 or int(i) == 9:
        for j in range(29):
            dates.append(datetime(year=start_year+j,month=int(i),day=1))
    else:
        for j in range(30):
            dates.append(datetime(year=start_year+j,month=int(i),day=1))

    dx_forecasts_second_attempt.append(xr.DataArray(data=only_july_data_reordered[i],coords=[dates,only_july_data_reordered[i][0].flow],dims=['forecast_date','ensemble_year']))

In [9]:
dx_forecasts_second_attempt[11]

<xarray.DataArray (forecast_date: 30, ensemble_year: 30)>
array([[23780.8661157 , 27886.61157025, 28474.61157025, 21589.17024793,
        29773.62644628, 23518.98842975, 22226.97520661, 28969.1107438 ,
        24701.19669421, 29428.58181818, 25742.7768595 , 23915.88099174,
        22675.45785124, 23541.24297521, 22782.18842975, 25524.33719008,
        25016.4892562 , 23580.89256198, 22254.18842975, 21825.81818182,
        26011.47768595, 27463.99338843, 26830.90909091, 27451.25950413,
        31504.72066116, 24882.86280992, 23464.87933884, 22670.49917355,
        21971.50413223, 23189.11735537],
       [14512.52231405, 17794.23471074, 19983.41157025, 13438.45289256,
        19871.08760331, 14736.77355372, 13520.15206612, 19019.20661157,
        15092.21157025, 18909.0446281 , 15698.89586777, 15134.18181818,
        13939.61652893, 14109.85785124, 14002.94876033, 16200.03966942,
        15534.04958678, 14691.55041322, 13823.66280992, 13396.20495868,
        16709.29586777, 17673.85785124, 16986.10909091, 17271.66942149,
        21150.6446281 , 15935.68264463, 14291.08760331, 13694.20165289,
        13527.05454545, 14397.40165289],
       [42435.49090909, 45767.22644628, 45467.16694215, 38063.66280992,
        47406.6446281 , 42319.21983471, 40148.3107438 , 49542.2677686 ,
        45135.92727273, 49103.36528926, 44455.83471074, 41935.00165289,
        40925.63305785, 42988.46280992, 41708.21157025, 44793.95702479,
...
         8057.9107438 ,  7822.82975207,  7679.42479339,  7732.22479339,
         8541.40165289,  8558.04297521,  9434.24132231,  9098.06280992,
         9891.6892562 ,  8275.18016529,  8048.56859504,  7715.52396694,
         7567.35867769,  7694.38016529],
       [55039.77520661, 58127.36528926, 58887.17355372, 50435.40495868,
        60464.68760331, 55452.21818182, 52950.98181818, 63763.2       ,
        58454.91570248, 62166.66446281, 57757.26942149, 54733.34876033,
        53902.1553719 , 56847.11404959, 55402.43305785, 58499.04793388,
        57694.7107438 , 54243.19338843, 53078.63801653, 53217.10413223,
        58190.16198347, 61029.24297521, 60773.39504132, 60299.40495868,
        64344.81322314, 57874.29421488, 55103.72231405, 54175.47768595,
        52402.84958678, 53285.69256198],
       [14103.23305785, 17152.68099174, 20990.55867769, 13810.51239669,
        19857.10413223, 14514.4661157 , 13867.87438017, 18065.15702479,
        14669.85123967, 18020.52892562, 14765.09752066, 14780.82644628,
        13865.49421488, 14641.68595041, 14379.92727273, 15957.81818182,
        14959.39834711, 14090.73719008, 14262.66446281, 13936.81983471,
        15858.12892562, 16932.63471074, 16534.35371901, 16278.76363636,
        20048.82644628, 16023.11404959, 14193.85785124, 13867.89421488,
        13839.82809917, 14184.85289256]])
Coordinates:
  * forecast_date  (forecast_date) datetime64[ns] 1991-07-01 ... 2020-07-01
  * ensemble_year  (ensemble_year) int64 1991 1992 1993 1994 ... 2018 2019 2020

In [10]:
11989.52*CFSDAY2ACFT

23780.86611570248

In [11]:
dx_forecasts_all[11]

<xarray.DataArray (forecast_date: 30, ensemble_year: 30)>
array([[125049.37190083, 129155.11735537, 129743.11735537,
        122857.67603306, 131042.1322314 , 124787.49421488,
        123495.48099174, 130237.61652893, 125969.70247934,
        130697.08760331, 127011.28264463, 125184.38677686,
        123943.96363636, 124809.74876033, 124050.69421488,
        126792.84297521, 126284.99504132, 124849.39834711,
        123522.69421488, 123094.32396694, 127279.98347107,
        128732.49917355, 128099.41487603, 128719.76528926,
        132773.22644628, 126151.36859504, 124733.38512397,
        123939.00495868, 123240.00991736, 124457.6231405 ],
       [107138.98512397, 110420.69752066, 112609.87438017,
        106064.91570248, 112497.55041322, 107363.23636364,
        106146.61487603, 111645.66942149, 107718.67438017,
        111535.50743802, 108325.35867769, 107760.6446281 ,
        106566.07933884, 106736.32066116, 106629.41157025,
        108826.50247934, 108160.51239669, 107318.01322314,
        106450.12561983, 106022.6677686 , 109335.75867769,
        110300.32066116, 109612.57190083, 109898.1322314 ,
        113777.10743802, 108562.14545455, 106917.55041322,
        106320.66446281, 106153.51735537, 107023.86446281],
...
       [181897.33614069, 184984.92622334, 185744.7344878 ,
        177292.96589276, 187322.24853739, 182309.7791159 ,
        179808.54275226, 190620.76093408, 185312.47663656,
        189024.22539689, 184614.83035557, 181590.90969441,
        180759.71630598, 183704.67498367, 182259.99399193,
        185356.60886797, 184552.27167788, 181100.75432251,
        179936.19895061, 180074.66506631, 185047.72291755,
        187886.80390929, 187630.9559754 , 187156.96589276,
        191202.37415722, 184731.85514896, 181961.28324813,
        181033.03862003, 179260.41052086, 180143.25349607],
       [109034.47269927, 112083.92063316, 115921.79831911,
        108741.75203812, 114788.34377365, 109445.70575712,
        108799.11402159, 112996.39666621, 109601.09088109,
        112951.76856704, 109696.33716208, 109712.0660877 ,
        108796.7338563 , 109572.92559183, 109311.16691415,
        110889.05782324, 109890.63798853, 109021.9768315 ,
        109193.90410423, 108868.05947613, 110789.36856704,
        111863.87435217, 111465.59336043, 111210.00327779,
        114980.0660877 , 110954.35369101, 109125.09749266,
        108799.1338563 , 108771.06774059, 109116.09253398]])
Coordinates:
  * forecast_date  (forecast_date) datetime64[ns] 1991-07-01 ... 2020-07-01
  * ensemble_year  (ensemble_year) int64 1991 1992 1993 1994 ... 2018 2019 2020

### DataFrame of Calculated Values

In [20]:
dataframes = []
datesList = []

#looping through each xarray
for i in dx_forecasts_all:

    for j in i:
        
        #making list of forecast date (repetitive list of the same date)
        forecastDate = [j.forecast_date.values.astype('datetime64[D]') for k in j.data]
        
        #adding the forecast date once to datesList
        datesList.append(j.forecast_date.values.astype('datetime64[D]'))
        
        #saving current forecast date
        Date2 = j.forecast_date.values.astype('datetime64[D]')

        #saving year of forecast date
        DateYear = j.forecast_date.values.astype('datetime64[Y]')
        
        #making list of ensemble years from 1991 to 2020
        ensembleYear = [str(1991+i) for i in range(30)]
        
        actualObsSum = []
        addObsSum = []
        totalForecastSum = []
        
        # if month is august or later, add one to the year since it is in new water year
        if pd.to_datetime(Date2).month >= 8:
            yr = str(int(str(DateYear))+1)
            
            #filter dictionary of dataframes containing the forecasts and sum over the runoff period
            baseForecastSum = dfList[yr][str(Date2)][yr+'-04-01':yr+"-07-31"].sum(axis=0)
            
            #list of 0 since there is no additional observational data to add
            addObsSum = [0 for i in ensembleYear]
            
            actualObsSum = [float(f_obsFlow[yr+'-04-01':yr+"-07-31"].sum()) for i in ensembleYear]
            
        # if month is april or earlier keep the year the same and do same as previous if statement
        elif pd.to_datetime(Date2).month < 5:
            yr = str(int(str(DateYear)))
            baseForecastSum = dfList[yr][str(Date2)][yr+'-04-01':yr+"-07-31"].sum(axis=0)
            addObsSum = [0 for i in ensembleYear]
            actualObsSum = [float(f_obsFlow[yr+'-04-01':yr+"-07-31"].sum()) for i in ensembleYear]
        #remaining are only months within the runoff period that will see additional observational data
        else:
            yr = str(int(str(DateYear)))
            baseForecastSum = dfList[yr][str(Date2)][yr+'-04-01':yr+"-07-31"].sum(axis=0)
            
            #retrieve the sum of observational data from beginning of runoff period to one day prior to the forecast date
            addObsSum = [float(f_obsFlow[yr+'-04-01':str(Date2-1)].sum()) for i in ensembleYear]
            actualObsSum = [float(f_obsFlow[yr+'-04-01':yr+"-07-31"].sum()) for i in ensembleYear]

        # adding base forecast sum with observational sum
        totalForecastSum = [baseForecastSum_i+addObsSum_i for baseForecastSum_i,addObsSum_i in zip(baseForecastSum,addObsSum)]
            
        #gathering percent difference between total forecasted sum and observational sum
        percDiff = [((totalForecastSum_i - actualObsSum_i)/actualObsSum_i)*100 for totalForecastSum_i, actualObsSum_i in zip(totalForecastSum,actualObsSum)]
        
        #getting absolute value of difference between forecasted sum and actual sum
        err = [abs(totalForecastSum_i - actualObsSum_i) for totalForecastSum_i, actualObsSum_i in zip(totalForecastSum,actualObsSum)]
        
        #creating dataframe
        df = pd.DataFrame({ 'Base Forecast Sum':baseForecastSum,
                            'Additional Observed Sum':addObsSum,
                            'Total Forecasted Sum':totalForecastSum,
                            'Actual Sum':actualObsSum,
                            'Error':err,
                            '% Difference':percDiff})
        
        #multiplying all columns but % difference by the CFS to ACFT per day conversion
        df[list(df.keys()[:-1])] = df[list(df.keys()[:-1])]*CFSDAY2ACFT
        dataframes.append(df)
statsDF = pd.concat(dataframes, keys=datesList)        

In [21]:
statsDF.loc['1991-07-01']

,Base Forecast Sum,Additional Observed Sum,Total Forecasted Sum,Actual Sum,Error,% Difference
1991,23780.866116,101268.505785,125049.371901,121103.119339,3946.252562,3.258589
1992,27886.611570,101268.505785,129155.117355,121103.119339,8051.998017,6.648877
1993,28474.611570,101268.505785,129743.117355,121103.119339,8639.998017,7.134414
1994,21589.170248,101268.505785,122857.676033,121103.119339,1754.556694,1.448812
1995,29773.626446,101268.505785,131042.132231,121103.119339,9939.012893,8.207066
1996,23518.988430,101268.505785,124787.494215,121103.119339,3684.374876,3.042345
1997,22226.975207,101268.505785,123495.480992,121103.119339,2392.361653,1.975475
1998,28969.110744,101268.505785,130237.616529,121103.119339,9134.497190,7.542743
1999,24701.196694,101268.505785,125969.702479,121103.119339,4866.583140,4.018545
2000,29428.581818,101268.505785,130697.087603,121103.119339,9593.968264,7.922148


In [22]:
dataframes = []
datesList = []

#looping through each xarray
for i in dx_forecasts_second_attempt:

    for j in i:
        
        
        
        #making list of forecast date (repetitive list of the same date)
        forecastDate = [j.forecast_date.values.astype('datetime64[D]') for k in j.data]
        
        #adding the forecast date once to datesList
        datesList.append(j.forecast_date.values.astype('datetime64[D]'))
        
        #saving current forecast date
        Date2 = j.forecast_date.values.astype('datetime64[D]')

        #saving year of forecast date
        DateYear = j.forecast_date.values.astype('datetime64[Y]')
        
#         print(forecastDate[0],Date2,DateYear)
#         break
        
        #making list of ensemble years from 1991 to 2020
        ensembleYear = [str(1991+i) for i in range(30)]
        
        actualObsSum = []
        addObsSum = []
        totalForecastSum = []
        
        # if month is august or later, add one to the year since it is in new water year
        if pd.to_datetime(Date2).month >= 8:
            yr = str(int(str(DateYear))+1)
            
            #filter dictionary of dataframes containing the forecasts and sum over the runoff period
            baseForecastSum = dfList[yr][str(Date2)][yr+'-04-01':yr+"-07-31"].sum(axis=0)
            
            #list of 0 since there is no additional observational data to add
            addObsSum = [0 for i in ensembleYear]
            
            actualObsSum = [float(f_obsFlow[yr+'-04-01':yr+"-07-31"].sum()) for i in ensembleYear]
            
        # if month is april or earlier keep the year the same and do same as previous if statement
        elif pd.to_datetime(Date2).month < 5:
            yr = str(int(str(DateYear)))
            baseForecastSum = dfList[yr][str(Date2)][yr+'-04-01':yr+"-07-31"].sum(axis=0)
            addObsSum = [0 for i in ensembleYear]
            actualObsSum = [float(f_obsFlow[yr+'-04-01':yr+"-07-31"].sum()) for i in ensembleYear]
        #remaining are only months within the runoff period that will see additional observational data
        else:
            yr = str(int(str(DateYear)))
            baseForecastSum = dfList[yr][str(Date2)][yr+'-04-01':yr+"-07-31"].sum(axis=0)
            
            #retrieve the sum of observational data from beginning of runoff period to one day prior to the forecast date
#             addObsSum = [0 for i in ensembleYear]
            actualObsSum = [float(f_obsFlow[yr+'-'+str(pd.to_datetime(Date2).month).zfill(2)+'-01':yr+"-07-31"].sum()) for i in ensembleYear]
            
        #gathering percent difference between total forecasted sum and observational sum
        percDiff = [((baseForecastSum_i - actualObsSum_i)/actualObsSum_i)*100 for baseForecastSum_i, actualObsSum_i in zip(baseForecastSum,actualObsSum)]
        
        #getting absolute value of difference between forecasted sum and actual sum
        err = [abs(baseForecastSum_i - actualObsSum_i) for baseForecastSum_i, actualObsSum_i in zip(baseForecastSum,actualObsSum)]
        
        #creating dataframe
        df = pd.DataFrame({ 'Forecast Sum':baseForecastSum,
                            'Actual Sum':actualObsSum,
                            'Error':err,
                            '% Difference':percDiff})
        
        #multiplying all columns but % difference by the CFS to ACFT per day conversion
        df[list(df.keys()[:-1])] = df[list(df.keys()[:-1])]*CFSDAY2ACFT
        dataframes.append(df)
statsDF_second_attempt = pd.concat(dataframes, keys=datesList)        

In [23]:
statsDF_second_attempt.loc['1991-07-01']

,Forecast Sum,Actual Sum,Error,% Difference
1991,23780.866116,19834.613554,3946.252562,19.895787
1992,27886.611570,19834.613554,8051.998017,40.595689
1993,28474.611570,19834.613554,8639.998017,43.560203
1994,21589.170248,19834.613554,1754.556694,8.845933
1995,29773.626446,19834.613554,9939.012893,50.109436
1996,23518.988430,19834.613554,3684.374876,18.575481
1997,22226.975207,19834.613554,2392.361653,12.061549
1998,28969.110744,19834.613554,9134.497190,46.053316
1999,24701.196694,19834.613554,4866.583140,24.535810
2000,29428.581818,19834.613554,9593.968264,48.369827


In [24]:
statsDF_second_attempt.loc['1991-07-01']['Forecast Sum'].mean()

25088.34115702479

In [25]:
clim_obs = []
for year in ensembleYear:
    clim_obs.append(float(f_obsFlow[year+'-07-01':year+"-07-31"].sum())*CFSDAY2ACFT)

In [26]:
clim_obs

[19834.613553718995,
 19649.4029752066,
 43945.72958677683,
 10584.126942148743,
 83939.83933884287,
 25201.33090909091,
 28525.299173553703,
 33205.28925619834,
 24787.547107438008,
 13091.599338842969,
 11404.901157024791,
 3716.9573553719,
 15468.386776859492,
 12085.560991735529,
 19695.00268601652,
 17335.082622347094,
 19392.73406995041,
 38859.83094247932,
 24149.189102082637,
 13848.856844033045,
 66644.27609829431,
 7617.406520528915,
 12538.565049732359,
 26104.444848734118,
 26580.88283226445,
 18849.089397421478,
 20502.143772099167,
 6132.368544396692,
 63906.5041638346,
 10285.855793256196]

In [27]:
9999.951*CFSDAY2ACFT

19834.613553719006

In [28]:
errDF = pd.DataFrame(statsDF_second_attempt.groupby(level=0).Error.mean())
errDF.groupby(errDF.index.month).mean().reindex([8,9,10,11,12,1,2,3,4,5,6,7])

,Error
8,43550.792848
9,43655.944080
10,43193.158622
11,41942.763760
12,41556.187507
1,38046.281249
2,33926.605157
3,29881.203853
4,23476.274025
5,17399.280566


### Calculating CRPS & CRPSS -- Manual Method -- Original Workflow

- Observational volumes added to Forecasted (depending on month in runoff period)
- Ensemble year corresponding to forecast year is excluded  
- Analyzing full runoff period 

In [37]:
def CRPS_Calc(_2dXarray, all_runoff_months = True):
    
    # lists to be populated by mean values of CRPS and CRPSS from each month
    crpsMean = []
    crpssMean = []
    
    # list of ensemble years
    ensembleYear = [str(1991+i) for i in range(30)]

    # iterating through each collection of forecast months (forecasts created in October,November, December, etc)
    # j is an xarray containing forecasts only from specific months
    for j in _2dXarray:
        
        # Retrieving month of data being analyzed
        _month = str(j.forecast_date.values.astype('datetime64[D]')[0].astype('datetime64[M]').astype(int) % 12 + 1)
        print("Month:",_month)
        
#         if _month != '7':
#             continue
        
        # establishing blank lists to be populated for each month's forecasts
        crpsYearly = []
        crpssYearly = []
        clim_obs = []
        
        # collecting climatological data 
        for year in ensembleYear:
            
            # checking if method takes all runoff months into consideration
            if all_runoff_months:
                
                # if yes, sum observed inflows over entire runoff period
                clim_obs.append(float(f_obsFlow[year+'-04-01':year+"-07-31"].sum())*CFSDAY2ACFT)
                
            else:
                
                # if no and month falls within runoff period
                if int(_month) > 4 and int(_month) <= 7:
                    
                    # sum observed inflows from only that month to the end of the runoff period
                    clim_obs.append(float(f_obsFlow[year+'-'+_month.zfill(2)+'-01':year+"-07-31"].sum())*CFSDAY2ACFT)
                else:
                    
                    # if month falls outside of runoff period sum observed inflows over entire runoff period
                    clim_obs.append(float(f_obsFlow[year+'-04-01':year+"-07-31"].sum())*CFSDAY2ACFT)
            
        
        # climatology list has now been gathered 
            # reflects observed inflows for respective time periods to that of the forecasted data 
        
        # looping through each forecast year within the monthly data
        for yrAdd,k in enumerate(j.data):

            # determining active year
            year = str(1991 + yrAdd)

            if all_runoff_months:
            
                # retrieving sum of observed flows in runoff period for given water year
                obs = float(f_obsFlow[year+'-04-01':year+"-07-31"].sum())*CFSDAY2ACFT
            
            else:
                
                # retrieving sum of observed flows in runoff period starting in the forecasted month for given water year
                if int(_month) > 4 and int(_month) <= 7:
                    obs = float(f_obsFlow[year+'-'+_month.zfill(2)+'-01':year+"-07-31"].sum())*CFSDAY2ACFT
                else:
                    
                    # retrieving sum of observed flows in runoff period for given water year
                    obs = float(f_obsFlow[year+'-04-01':year+"-07-31"].sum())*CFSDAY2ACFT          


            # removing the ensemble year matching the water year in question
            forecastEnsemble = np.delete(k,yrAdd)
            climEnsemble = np.delete(clim_obs,yrAdd)

            # calculating bias corrected forecast CRPS 
            crps = CRPS.CRPS(forecastEnsemble,obs).compute()[0]
            
            # append CRPS value for this water year
            crpsYearly.append(crps)

            # calculating bias corrected climatology CRPS
            crpsClim = CRPS.CRPS(climEnsemble,obs).compute()[0]

            # calculating CRPSS
            crpss = 1-(crps/crpsClim)

            # append CRPSS value for this water year
            crpssYearly.append(crpss)

    #     taking the mean CRPS and CRPSs for each month
        crpsMean.append(np.mean(crpsYearly))
        crpssMean.append(np.mean(crpssYearly))
        
    return crpsMean, crpssMean

In [38]:
crps_all, crpss_all = CRPS_Calc(dx_forecasts_all)

Month: 8
Month: 9
Month: 10
Month: 11
Month: 12
Month: 1
Month: 2
Month: 3
Month: 4
Month: 5
Month: 6
Month: 7


In [39]:
102363.408*CFSDAY2ACFT

203034.8588429752

### Calculating CRPS & CRPSS New Way

- No observational data added to forecast
- Ensemble year corresponding to forecast year is excluded  
- only observing month of July

In [40]:
crps_2,crpss_2 = CRPS_Calc(dx_forecasts_second_attempt,False)

Month: 8
Month: 9
Month: 10
Month: 11
Month: 12
Month: 1
Month: 2
Month: 3
Month: 4
Month: 5
Month: 6
Month: 7


In [41]:
np.mean([3778.729679936337, 2831.2510639635934, 952.3004011359945, 3898.5382596476234, 2691.7629040594484, 762.7943695521857, 5216.407476341639, 2470.4477746877415, 6809.614744351971, 345.1161556981534, 2068.235286209848, 1481.9856817444806, 3896.2575014003532, 685.8056732933136, 722.4655474751994, 2079.3825336009554, 840.1052924313226, 19696.91135368523, 13941.54351257327, 7334.792931421203, 1508.6474343439918, 2078.416046771778, 2731.467485325279, 949.7636588055526, 3965.9231129220702, 632.5464936471963, 1665.7799375951847, 1830.1969807062442, 4896.371977419457, 4327.6586887162675])

3569.707331982097

In [42]:
crps_2

[21890.48211033595,
 22053.763986960483,
 23407.350298186022,
 23850.081528238232,
 23302.595570862937,
 21293.52980405487,
 19042.683921449636,
 17190.013999600822,
 13799.83161134066,
 10503.074735355458,
 5912.221106896309,
 3569.707331982097]

In [43]:
crpss_2

[0.14817580951092016,
 0.14023309576254117,
 0.11814183989435209,
 0.054516726752144556,
 0.016993103891077412,
 0.11644097423089388,
 0.22205424431693171,
 0.2888423812863953,
 0.39255265599435646,
 0.4868668235377065,
 0.6646912827834633,
 0.37069302404044774]